In [ ]:
pip install pyspark

In [ ]:
# Importation des bibliothèques
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
#. Initialiser Spark Session
spark = SparkSession.builder \
    .appName("Analyse des maladies cardiaques") \
    .getOrCreate()

In [ ]:
# 1. Collecte de Données
# Charger le fichier CSV (le fichier uploadé dans Colab)
file_path = "heart.csv"
data = spark.read.csv(file_path, header=True, inferSchema=True)
# Aperçu des données
data.show(5)
data.printSchema()

In [ ]:
# 2. Préparation des Données
# a) Gérer les valeurs manquantes
data = data.na.fill(0)  # Remplacer les valeurs nulles par 0

# b) Supprimer les doublons
data = data.dropDuplicates()

# c) Renommer les colonnes pour une meilleure lisibilité
for col_name in data.columns:
    data = data.withColumnRenamed(col_name, col_name.lower())

# Valider les changements
data.show(5)

In [ ]:
# 3. Stockage des Données
# Sauvegarder les données nettoyées dans un fichier Parquet (simulation de HDFS)
data.write.parquet("/content/cleaned_data.parquet", mode="overwrite")

In [ ]:
# 4. Analyse Exploratoire des Données (EDA)
# a) Utiliser Spark SQL
# Créer une vue temporaire pour exécuter des requêtes SQL
data.createOrReplaceTempView("heart_data")

# Exemple : compter les entrées par groupe "target"
target_count = spark.sql("SELECT target, COUNT(*) as count FROM heart_data GROUP BY target")
target_count.show()

# b) Visualisation avec Matplotlib et Seaborn
# Convertir les données Spark en Pandas pour la visualisation
pandas_df = target_count.toPandas()

# Visualisation
sns.barplot(data=pandas_df, x="target", y="count")
plt.title("Distribution des cibles (target)")
plt.show()

In [ ]:
# 5. Modélisation et Analyse
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

# a) Préparation des données pour le clustering
feature_columns = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']  # Exemple de colonnes
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
cluster_data = assembler.transform(data)

# b) Appliquer l'algorithme de clustering KMeans
kmeans = KMeans(k=3, seed=1, featuresCol="features")
model = kmeans.fit(cluster_data)
clusters = model.transform(cluster_data)

# c) Afficher les résultats du clustering
clusters.select("age", "trestbps", "chol", "prediction").show(5)

In [ ]:
# 6. Visualisation des Résultats
# Exemple de visualisation des clusters
cluster_pandas = clusters.select("age", "trestbps", "chol", "prediction").toPandas()
sns.scatterplot(data=cluster_pandas, x="age", y="chol", hue="prediction", palette="viridis")
plt.title("Clusters basés sur l'âge et le cholestérol")
plt.show()

In [ ]:
clusters.select("age", "trestbps", "chol", "prediction").write.csv("/content/heart.csv", header=True, mode="overwrite")
